In [ ]:
import geopandas as gpd
import pandas as pd
import requests
import math
import numpy
from datetime import datetime
from shapely import wkt
from shapely.errors import WKTReadingError
from shapely.geometry import mapping, shape
from shapely import Polygon
import warnings
warnings.filterwarnings("ignore")
import re
import numpy as np
from shapely.validation import make_valid

In [ ]:
df = pd.read_excel(r"C:\Users\dorito\Documents\Levantamiento Asociados mayores de 4 ha faltantes 10 01 2025.xlsx")

In [ ]:
lista_cedulas = df["CEDULA"].to_list()

In [ ]:
login = {
    "username":"Dorito",
    "password":"Portador123"
}
# Crear una sesión para mantener la cookie
session = requests.Session()
response = session.post("https://192.168.179.3/api/user/login/", login, verify=False)

In [ ]:
def get_data_paginated(url):
    login = {
        "username":"Dorito",
        "password":"Portador123"
    }
    # Crear una sesión para mantener la cookie
    session = requests.Session()
    response = session.post("https://192.168.179.3/api/user/login/", login, verify=False)
    datos = []
    while url:
        response = session.get(url, verify=False)
        info = response.json()
        datos.extend(info["results"])
        url = info["next"]
    return datos

def get_data_not_paginated(url):
    login = {
        "username":"Dorito",
        "password":"Portador123"
    }
    # Crear una sesión para mantener la cookie
    session = requests.Session()
    response = session.post("https://192.168.179.3/api/user/login/", login, verify=False)
    response = session.get(url, verify=False)
    info = response.json()
    return info


def parse_geom_safe(wkt_string):
    try:
        if isinstance(wkt_string, str) and "SRID=" in wkt_string:
            return wkt.loads(wkt_string.split(";", 1)[1])
    except:
        pass
    return None  # simplemente ignora lo inválido

In [ ]:
gdf_fincas_mancipe = gpd.read_file(r"C:\Users\dorito\Downloads\fincas mancipe 2 nov.geojson")
gdf_lotes_mancipe = gpd.read_file(r"C:\Users\dorito\Downloads\Lote Mancipe 2 Nov.geojson")
# gdf_conservacion_mancipe = gpd.read_file(r"C:\Users\dorito\Documents\arreglando bd\Poligonos_Alvaro\Conservacion.shp")
# gdf_infraestructra_mancipe = gpd.read_file(r"C:\Users\dorito\Documents\arreglando bd\Poligonos_Alvaro\Infraestructura.shp")

In [ ]:
# --- FINCAS ---
gdf_fincas_mancipe["CEDULA"] = pd.to_numeric(
    gdf_fincas_mancipe["CEDULA"], errors="coerce"
).astype("Int64")

# --- LOTES ---
gdf_lotes_mancipe["CEDULA"] = pd.to_numeric(
    gdf_lotes_mancipe["CEDULA"], errors="coerce"
).astype("Int64")


In [ ]:
gdf_fincas = gdf_fincas_mancipe.loc[gdf_fincas_mancipe["CEDULA"].isin(lista_cedulas)]
gdf_lotes = gdf_lotes_mancipe.loc[gdf_lotes_mancipe["CEDULA"].isin(lista_cedulas)]

In [ ]:
gdf_lotes.columns
gdf_lotes["TIEMPO"]

In [ ]:
gdf_fincas["FECHA_ULTIMA_VISITA"] = (
    pd.to_datetime(gdf_fincas["FECHA_ULTIMA_VISITA"], errors='coerce')
    .dt.strftime('%Y-%m-%d')
    .fillna('')
)
gdf_lotes["TIEMPO"] = (
    pd.to_datetime(gdf_lotes["TIEMPO"], errors='coerce')
    .dt.strftime('%Y-%m-%d')
    .fillna('')
)


In [ ]:
import pandas as pd
import requests
def safe_cedula(x, default=1):
    """Convierte a int si es posible, si no devuelve 1 por defecto."""
    if pd.isna(x) or str(x).strip() == "":
        return default
    try:
        return int(float(x))  # si viene como '12345.0'
    except Exception:
        return default

def enviar_poligono(row, session, mongo_atribute, url):
    elementos = []  # lo que falla
    try:
        documento = safe_cedula(row["CEDULA"])
        respuesta = {
            "documento_productor":int(documento),
            "poligono":str(row["geometry"]),
            "mongo_atribute": mongo_atribute
        }
        # petición al servidor
        resp = session.post(url, json=respuesta, verify=False)
        elementos = {
            "payload": respuesta,
            "status_code": resp.status_code,
            "response_text": resp.text
        }
    except:
        elementos = {
            "payload": respuesta
        }

    return elementos


In [ ]:
gdf_fincas.columns

In [ ]:
SUBIDAS = []
for i, row in gdf_fincas.iterrows():
    if pd.notna(row["FECHA_ULTIMA_VISITA"]) and str(row["FECHA_ULTIMA_VISITA"]).strip() != "":
        fecha_visita = row["FECHA_ULTIMA_VISITA"]
    else:
        fecha_visita = ""

    if pd.notna(row["NOMBRE_FINCA"]) and str(row["NOMBRE_FINCA"]).strip() != "":
        NOMBRE_FINCA = row["NOMBRE_FINCA"]
    else:
        NOMBRE_FINCA = ""
    mongo_atribute = {
        "nombre_finca": NOMBRE_FINCA,
        "fecha_visita": fecha_visita
    }
    elementos = enviar_poligono(row, session, mongo_atribute, "https://192.168.179.3/api/v1/poligonos_fincas/")
    SUBIDAS.append(elementos)

In [ ]:
df_fincas_subidas = pd.DataFrame(SUBIDAS)

In [ ]:
df_fincas_subidas.to_excel("./fincas_subidas_mancipe_octubre.xlsx")

In [ ]:
gdf_lotes["COD_LOTE"] = gdf_lotes["COD_LOTE"].apply(
    lambda x: int(str(x).split(" ")[-1]) if pd.notna(x) and str(x).strip() != "" else None
)


In [ ]:
gdf_lotes.columns

In [ ]:
# Asegurar VARIEDAD en mayúsculas
gdf_lotes["VARIEDAD"] = gdf_lotes["VARIEDAD"].astype(str).str.upper()

# Crear la nueva columna 'variedades' con el valor correcto
gdf_lotes["variedades"] = gdf_lotes.apply(
    lambda row: row["VARIEDAD"] if pd.isna(row["variedades_2"]) or str(row["variedades_2"]).strip() == "" else row["variedades_2"],
    axis=1
)


In [ ]:
gdf_lotes['ESTADO_PYTHON'].unique()

In [ ]:
gdf_lotes['ESTADO_PYTHON'] = gdf_lotes['ESTADO_PYTHON'].map({
    "produccion": 1,
    "levante": 0,
    None: 0
})

In [ ]:
gdf_lotes["SUB_O"].unique()

In [ ]:
def limpiar_nan(obj):
    if isinstance(obj, dict):
        return {k: limpiar_nan(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [limpiar_nan(v) for v in obj]
    elif isinstance(obj, float) and (math.isnan(obj) or math.isinf(obj)):
        return None
    else:
        return obj
    
def safe_int(x, default=0):
    """Convierte a int si es posible, si no devuelve default."""
    try:
        if pd.notna(x) and str(x).strip() != "":
            return int(float(x))  # por si viene como '12.0'
        else:
            return default
    except (ValueError, TypeError):
        return default

SUBIDAS = []

for i, row in gdf_lotes.iterrows():
    if pd.notna(row["SUB_O"]) and str(row["SUB_O"]).strip() != "":
        subtipo_operacion = int(row["SUB_O"])
    else:
        subtipo_operacion = 2
    mongo_atribute = {
        "fecha_visita": "",
        "observaciones": "",
        "descripcion": "",
        "numero_lote": safe_int(row["COD_LOTE"]),
        "descripcion_lote": "",
        "variedad": row["variedades"],
        "distancia_surcos": safe_int(row["D_CALLE"]),
        "distancia_plantas": safe_int(row["D_PLANTA"]),
        "densidad": safe_int(row["DENSIDAD"]),
        "numero_plantas": safe_int(row["ARBOLES"]),
        "gramos_plantas": 500,
        "kg_produccion": safe_int(row["ARBOLES"]) / 2,
        "fecha_actividad": "",
        "produccion": safe_int(row["ESTADO_PYTHON"]),
        "estado": "BUENO",
        "subtipo_operacion": subtipo_operacion,
    }
    
    elementos = enviar_poligono(row, session, mongo_atribute, "https://192.168.179.3/api/v1/poligonos_lotes/")
    SUBIDAS.append(elementos)
    # print(mongo_atribute)
print("Elementos_enviados:", len(SUBIDAS))

In [ ]:
len(gdf_lotes)

In [ ]:
df_lotes_subidos = pd.DataFrame(SUBIDAS)

In [ ]:
df_lotes_subidos.to_excel("./lotes_subidos_octubre.xlsx")


In [ ]:
gdf_conservacion_no_solapa.columns

In [ ]:
SUBIDAS = []

for i, row in gdf_conservacion_no_solapa.iterrows():
    mongo_atribute = {
        "tipo_arbol": row["TIPO_ARBOL"]
    }
    mongo_atribute = limpiar_nan(mongo_atribute)
    elementos = enviar_poligono(row, session, mongo_atribute, "https://192.168.179.3/api/v1/poligonos_conservacion/")
    SUBIDAS.append(elementos)
    # print(mongo_atribute)
print("Elementos_enviados:", len(SUBIDAS))

In [ ]:
gdf_infraestructura_no_solapa.iloc[0].to_dict()

In [ ]:
import pandas as pd

cols = ["VIVIENDA", "ESTRUCTURA", "TIPO_INSTA"]

gdf_infraestructura_no_solapa["INSTALACIONES"] = gdf_infraestructura_no_solapa[cols].apply(
    lambda row: [str(x) for x in row if pd.notna(x) and str(x).strip() != ""],
    axis=1
)


In [ ]:
SUBIDAS = []

for i, row in gdf_infraestructura_no_solapa.iterrows():
    mongo_atribute = {
        "estructura": row["INSTALACIONES"]
    }
    mongo_atribute = limpiar_nan(mongo_atribute)
    elementos = enviar_poligono(row, session, mongo_atribute, "https://192.168.179.3/api/v1/poligonos_infraestructura/")
    SUBIDAS.append(elementos)
    # print(mongo_atribute)
print("Elementos_enviados:", len(SUBIDAS))

In [ ]:
df_borrar = pd.DataFrame(SUBIDAS)

In [ ]:
df_borrar.loc[df_borrar["status_code"] != 201]

In [ ]:
gdf_fincas_no_solapa.columns

In [ ]:
for i, row in gdf_fincas_no_solapa.iterrows():
    url_base_fincas = "https://192.168.179.3/api/v1/poligonos_fincas/"
    if not pd.isna(row["fecha_visi"]):
        fecha_visita = row["fecha_visi"].strftime("%Y-%m-%d")
    else:
        fecha_visita = ""
    
    if not pd.isna(row["documento"]):
        documento = int(float(row["documento"]))
    else:
        documento = 1

    respuesta_postgres = {
        "poligono": str(row["geometry"]),
        "documento_productor": documento 
    }

    respuesta_atributos = {
        "fecha_visita": fecha_visita,
        "nombre_finca": row["nombre_fin"],
        "descripcion": row["descripcio"]
    }
    if row["id"] == '1' or row["id"] == 1:
        respuesta_postgres["mongo_atribute"] = respuesta_atributos
        session.post(url=url_base_fincas, json=respuesta_postgres, verify=False)
    else:
        id_pol = row["id"]
        url_base_fincas_postgres = f"{url_base_fincas}{id_pol}/"
        session.patch(url_base_fincas_postgres, json=respuesta_postgres, verify=False)
        fecha_actual = datetime.now().strftime("%d-%m-%Y_00--00--00")
        url_base_fincas_mongo = f"{url_base_fincas}update/mongo_update/{id_pol}/{fecha_actual}/"
        mongo_atributes_json = {
            "mongo_atribute": respuesta_atributos
        }
        session.post(url_base_fincas_mongo, json=respuesta_postgres, verify=False)

In [ ]:
gdf_lotes.columns

In [ ]:
for i, row in gdf_lotes.iterrows():
    url_base_fincas = "https://192.168.179.3/api/v1/poligonos_lotes/"
    if not pd.isna(row["fecha_visi"]):
        fecha_visita = row["fecha_visi"].strftime("%Y-%m-%d")
    else:
        fecha_visita = ""

    if not pd.isna(row["fecha_acti"]):
        fecha_actividad = row["fecha_acti"].strftime("%Y-%m-%d")
    else:
        fecha_actividad = ""


    if not pd.isna(row["documento"]):
        documento = int(float(row["documento"]))
    else:
        documento = 1

    respuesta_postgres = {
        "poligono": str(row["geometry"]),
        "documento_productor": documento 
    }

    respuesta_atributos = {
        "numero_documento": str(row["documento"]),
        "nombre_productor": str(row['nom_prod']),
        "fecha_visita": fecha_visita,
        "area": safe_int(row['area']),
        "observaciones": "",
        "descripcion": "",
        "numero_lote": None,
        "descripcion_lote": "",
        "variedad": row['variedad'],
        "distancia_surcos": safe_int(row['distancia_']),
        "distancia_plantas": safe_int(row['distanci_1']),
        "densidad": safe_int(row['densidad']),
        "numero_plantas": safe_int(row['numero_pla']),
        "gramos_plantas": safe_int(row['gramos_pla']),
        "kg_produccion": safe_int(row['kg_producc']),
        "fecha_actividad": fecha_actividad,
        "produccion": row['produccion'],
        "estado_cultivo": row.get('estado_cul') if 'estado_cul' in row and pd.notnull(row['estado_cul']) else None,
        "subtipo_operacion": row['subtipo_op']
    }
    if row["id"] == '1' or row["id"] == 1:
        respuesta_postgres["mongo_atribute"] = respuesta_atributos
        session.post(url=url_base_fincas, json=respuesta_postgres, verify=False)
    else:
        id_pol = row["id"]
        url_base_fincas_postgres = f"{url_base_fincas}{id_pol}/"
        session.patch(url_base_fincas_postgres, json=respuesta_postgres, verify=False)
        fecha_actual = datetime.now().strftime("%d-%m-%Y_00--00--00")
        url_base_fincas_mongo = f"{url_base_fincas}update/mongo_update/{id_pol}/{fecha_actual}/"
        mongo_atributes_json = {
            "mongo_atribute": respuesta_atributos
        }
        session.post(url_base_fincas_mongo, json=respuesta_postgres, verify=False)